# VGG块
1. 使用3x3的卷积，深但窄的效果更好，选择更窄的3x3，而不是5x5和其他
2. 3x3的卷积，padding = 1， n层，m个通道
# 总结
1. VGG使用可重复使用的卷积块来构建深度卷积神经网络
2. 不同的卷积快个数和超参数可以得到不同复杂度的变种

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def vgg_block(num_convs, in_channels, out_channels) : 
    layers = []
    for _ in range(num_convs) : 
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [4]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
# 一共5块，每一次块相当于高宽减半， 输入维度是224*224， 除5次2之后就变成了7

def vgg(conv_arch) : 
    conv_blks = []
    in_channels = 1
    for(num_convs, out_channels) in conv_arch : 
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels
    return nn.Sequential(*conv_blks, nn.Flatten(),
                         nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(),
                         nn.Dropout(0.5), nn.Linear(4096, 4096), nn.ReLU(),
                         nn.Dropout(0.5), nn.Linear(4096, 10))

net = vgg(conv_arch)

In [10]:
#  观察每个层输出的形状
X = torch.randn(1, 1, 224, 224)
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'Output shape:\t', X.shape)

Sequential Output shape:	 torch.Size([1, 64, 112, 112])
Sequential Output shape:	 torch.Size([1, 128, 56, 56])
Sequential Output shape:	 torch.Size([1, 256, 28, 28])
Sequential Output shape:	 torch.Size([1, 512, 14, 14])
Sequential Output shape:	 torch.Size([1, 512, 7, 7])
Flatten Output shape:	 torch.Size([1, 25088])
Linear Output shape:	 torch.Size([1, 4096])
ReLU Output shape:	 torch.Size([1, 4096])
Dropout Output shape:	 torch.Size([1, 4096])
Linear Output shape:	 torch.Size([1, 4096])
ReLU Output shape:	 torch.Size([1, 4096])
Dropout Output shape:	 torch.Size([1, 4096])
Linear Output shape:	 torch.Size([1, 10])
